# TP Final - Gastón Vernhes

### IDEA DE TRADING

ESTRATEGIA CON CRUCE DE MEDIAS
    
    PASO 1: ver si hay alguna relacion del cruce de medias con los retornos
    PASO 2: ver si el MACD y el RSI mejoran la estrategia

### Indicadores a Utilizar

    PASO 1
    - Medias de 4, 8, 21, 42 periodos
    - Los cruces de las EMA de 4, 8 y 21 nos van a dar los puntos de entrada y salida
    - La EMA de 42 nos va a informar la tendencia actual más próxima
    
    PASO 2
    - MACD
    - RSI
 

### Codigo en una Linea

In [ ]:
# Eleccion del ticker 
import yfinance as yf
import pandas as pd

ticker = input('Ingrese el ticker: ')

datos = yf.download (ticker)['Adj Close']

# ---------------------------------------------------------------

# Calculo las medias a utilizar para la estrategia
adj_close = yf.download (ticker)['Adj Close']
ema4 = yf.download (ticker)['Adj Close'].rolling(4).mean()
ema8 = yf.download (ticker)['Adj Close'].rolling(8).mean()
ema21 = yf.download (ticker)['Adj Close'].rolling(21).mean()
ema42 = yf.download (ticker)['Adj Close'].rolling(42).mean()

# ---------------------------------------------------------------

# Pongo las medias dentro de un DataFrame y nombro las columnas
medias = pd.concat([adj_close, ema4, ema8, ema21, ema42], join = 'outer', axis = 1)
medias.columns = ('Adj Close','ema4', 'ema8', 'ema21', 'ema42')
medias.reset_index (inplace = True)   #reseteo el indice para que quede numerado desde el 0 hasta la ultima fila


# Condiciones de los Cruces de Medias: cuándo la media está por encima de la otra

medias ['ema4_ema8'] = medias['ema4'] > medias['ema8']     #CONDICION 1: es igual a True si ema4 > ema 8, sino False
medias ['ema8_ema21'] = medias['ema8'] > medias['ema21']    #CONDICION 2: es igual a True si ema8 > ema 21, sino False
medias ['ema21_ema42'] = medias['ema21'] > medias['ema42']   #CONDICION 3: es igual a True si ema21 > ema 42, sino False

# ---------------------------------------------------------------

# Armo la columna "ACCION", que va a indicar qué hacer con el activo a cada fecha (comprar, mantener, vender, quedarse afuera)

for i in range(1, len(medias)):   #se pone desde 1 xq se busca el elemento i-1, entonces con i=0 nos quedamos fuera del rango
    
    #Condicion para compra: ema8 cruce al alza a ema21, ema4 > ema 8, ema21 > ema42
    if (medias.loc[i,'ema8_ema21'] == True) and (medias.loc[i-1,'ema8_ema21'] == False) and (medias.loc[i,'ema4_ema8'] == True) and (medias.loc[i,'ema21_ema42'] == True):
        
        medias.loc[i,'accion'] = "Comprar"
        
    #Condicion para mantener el "long": ema8 se mantiene arriba de ema21, ema4 > ema 8, ema21 > ema42        
    elif (medias.loc[i,'ema8_ema21'] == True) and (medias.loc[i-1,'ema8_ema21'] == True) and (medias.loc[i,'ema4_ema8'] == True) and (medias.loc[i,'ema21_ema42'] == True):
        
        medias.loc[i,'accion'] = "Mantener"

    #Condicion para  vender: ema4 cruce a la baja a ema8, ema8 > ema 21, ema21 > ema42        
    elif (medias.loc[i,'ema4_ema8'] == False) and (medias.loc[i-1,'ema4_ema8'] == True) and (medias.loc[i,'ema8_ema21'] == True) and (medias.loc[i,'ema21_ema42'] == True):
        
        medias.loc[i,'accion'] = "Vender"
        
    else: 
        medias.loc[i,'accion'] = "No operar"

# ---------------------------------------------------------------

# Cuenta la cantidad de (comprar, vender, mantener, quedarse afuera) que se dieron
print (' ')
print ('La cantidad de dias con las siguientes acciones son:')
print(medias['accion'].value_counts())
print (' ')

# ---------------------------------------------------------------

# Calculo el RSI
import numpy as np

ruedas = 14

medias['dif'] = medias['Adj Close'].diff()
medias['win'] = np.where(medias['dif'] > 0, medias['dif'], 0)
medias['loss'] = np.where(medias['dif'] < 0, abs(medias['dif']), 0)
medias['ema_win'] = medias.win.ewm(alpha=1/ruedas).mean()
medias['ema_loss'] = medias.loss.ewm(alpha=1/ruedas).mean()
medias['rs'] = medias.ema_win / medias.ema_loss
medias['rsi'] = 100 - (100 / (1+medias.rs))
medias #= medias.reset_index().dropna().round(2)

# ---------------------------------------------------------------

# Calculo el MACD
slow = 26
fast = 12
suavizado = 9

medias['ema_fast'] = medias['Adj Close'].ewm(span=fast).mean()
medias['ema_slow'] = medias['Adj Close'].ewm(span=slow).mean()
medias['macd'] = medias.ema_fast - medias.ema_slow
medias['signal'] = medias.macd.ewm(span=suavizado).mean()
medias['histograma'] = medias.macd - medias.signal

pd.options.display.max_columns = None
medias #= medias.dropna().round(2)

# ---------------------------------------------------------------

# Tomo del DataFrame solo las filas con los dias de Compra y los de Venta

medias_cv = medias[(medias.accion == 'Vender') | (medias.accion == 'Comprar')]  # el simbolo "|" significo "o"
medias_cv.reset_index(inplace = True)


# ---------------------------------------------------------------

# Calculo la variacion entre las filas "Vender" que tiene como fila anterior a "Comprar"
#         Aclaracion: la compra es cuando ema8 > ema 21, pero la venta es cuando ema4 < ema8, por eso hay varias filas indicando vender pero hay menos filas indicando comprar

for i in range(1, len(medias_cv)):

    if (medias_cv.loc[i,'accion'] == 'Vender') and (medias_cv.loc[i-1,'accion'] == 'Comprar'):
        
        medias_cv.loc[i,'variacion'] = (medias_cv.loc[i,'Adj Close'] / medias_cv.loc[i-1,'Adj Close'] - 1) * 100
        


# ---------------------------------------------------------------

#Nos quedamos solo con las filas que tiene calculada la "variacion porcentual"

pd.options.display.max_rows = 20

medias_cv_parcial = medias_cv.dropna()


# ---------------------------------------------------------------

# Imprimimos el promedio, la mediana, el minimo y el maximo de los retornos

print ('Los datos de los retornos son: ')
print('El promedio es: ', round(medias_cv_parcial.variacion.mean(),2))
print('La mediana es: ', round(medias_cv_parcial.variacion.median(),2))
print('El mínimo es: ', round(medias_cv_parcial.variacion.min(),2))
print('El maximo es: ', round(medias_cv_parcial.variacion.max(),2))
print('*Los datos están en porcentaje')
print (' ')

# ---------------------------------------------------------------

#Hacemos un histograma de la variacion para ver como son las variaciones

print ('Grafico 1: Histograma - Retornos ')

medias_cv_parcial.variacion.plot(kind='hist', bins=100)

# ---------------------------------------------------------------


# Graficamos los Retornos con el RSI, para ver si hay alguna relacion

print ('Grafico 2: Relacion entre la variacion de los retornos y el RSI')

var_rsi= pd.concat([medias_cv_parcial['variacion'],medias_cv_parcial['rsi']], axis=1)

var_rsi.plot('variacion', 'rsi', s=1, kind='scatter')


# ---------------------------------------------------------------

# Graficamos los Retornos con el MACD, para ver si hay alguna relacion

print ('Grafico 3: Relacion entre la variacion de los retornos y el MACD')

var_macd= pd.concat([medias_cv_parcial['variacion'],medias_cv_parcial['histograma']], axis=1)

var_macd.plot('variacion', 'histograma', s=1, kind='scatter')

print (' ')

# ---------------------------------------------------------------


### ------------------------------------------------------------------------------------------------------

### Codigo separado en diferentes lineas

In [ ]:
import yfinance as yf
import pandas as pd

ticker = input('Ingrese el ticker: ')

datos = yf.download (ticker)['Adj Close']

In [ ]:
#Calculo las medias a utilizar para la estrategia
adj_close = yf.download (ticker)['Adj Close']
ema4 = yf.download (ticker)['Adj Close'].rolling(4).mean()
ema8 = yf.download (ticker)['Adj Close'].rolling(8).mean()
ema21 = yf.download (ticker)['Adj Close'].rolling(21).mean()
ema42 = yf.download (ticker)['Adj Close'].rolling(42).mean()


In [ ]:
#pongo las medias dentro de un DataFrame y nombro las columnas
medias = pd.concat([adj_close, ema4, ema8, ema21, ema42], join = 'outer', axis = 1)
medias.columns = ('Adj Close','ema4', 'ema8', 'ema21', 'ema42')
medias.reset_index (inplace = True)   #reseteo el indice para que quede numerado desde el 0 hasta la ultima fila


#Condiciones de los Cruces de Medias: cuándo la media está por encima de la otra

medias ['ema4_ema8'] = medias['ema4'] > medias['ema8']     #CONDICION 1: es igual a True si ema4 > ema 8, sino False
medias ['ema8_ema21'] = medias['ema8'] > medias['ema21']    #CONDICION 2: es igual a True si ema8 > ema 21, sino False
medias ['ema21_ema42'] = medias['ema21'] > medias['ema42']   #CONDICION 3: es igual a True si ema21 > ema 42, sino False


medias

In [ ]:
#Armo la columna "ACCION", que va a indicar qué hacer con el activo a cada fecha (comprar, mantener, vender, quedarse afuera)

for i in range(1, len(medias)):   #se pone desde 1 xq se busca el elemento i-1, entonces con i=0 nos quedamos fuera del rango
    
    #Condicion para compra: ema8 cruce al alza a ema21, ema4 > ema 8, ema21 > ema42
    if (medias.loc[i,'ema8_ema21'] == True) and (medias.loc[i-1,'ema8_ema21'] == False) and (medias.loc[i,'ema4_ema8'] == True) and (medias.loc[i,'ema21_ema42'] == True):
        
        medias.loc[i,'accion'] = "Comprar"
        
    #Condicion para mantener el "long": ema8 se mantiene arriba de ema21, ema4 > ema 8, ema21 > ema42        
    elif (medias.loc[i,'ema8_ema21'] == True) and (medias.loc[i-1,'ema8_ema21'] == True) and (medias.loc[i,'ema4_ema8'] == True) and (medias.loc[i,'ema21_ema42'] == True):
        
        medias.loc[i,'accion'] = "Mantener"

    #Condicion para  vender: ema4 cruce a la baja a ema8, ema8 > ema 21, ema21 > ema42        
    elif (medias.loc[i,'ema4_ema8'] == False) and (medias.loc[i-1,'ema4_ema8'] == True) and (medias.loc[i,'ema8_ema21'] == True) and (medias.loc[i,'ema21_ema42'] == True):
        
        medias.loc[i,'accion'] = "Vender"
        
    else: 
        medias.loc[i,'accion'] = "Quedarse Afuera"

        
medias

In [ ]:
#Cantidad de Oportunidades de compra 

print(medias['accion'].value_counts())

### Calculo el RSI

In [ ]:
#RSI
import numpy as np

ruedas = 14

medias['dif'] = medias['Adj Close'].diff()
medias['win'] = np.where(medias['dif'] > 0, medias['dif'], 0)
medias['loss'] = np.where(medias['dif'] < 0, abs(medias['dif']), 0)
medias['ema_win'] = medias.win.ewm(alpha=1/ruedas).mean()
medias['ema_loss'] = medias.loss.ewm(alpha=1/ruedas).mean()
medias['rs'] = medias.ema_win / medias.ema_loss
medias['rsi'] = 100 - (100 / (1+medias.rs))
medias #= medias.reset_index().dropna().round(2)

medias

### Calculo el MACD

In [ ]:
    
slow = 26
fast = 12
suavizado = 9

medias['ema_fast'] = medias['Adj Close'].ewm(span=fast).mean()
medias['ema_slow'] = medias['Adj Close'].ewm(span=slow).mean()
medias['macd'] = medias.ema_fast - medias.ema_slow
medias['signal'] = medias.macd.ewm(span=suavizado).mean()
medias['histograma'] = medias.macd - medias.signal

pd.options.display.max_columns = None
medias #= medias.dropna().round(2)

#### Me quedo con las filas de la tabla que tengan en la columna "Accion" las palabras "Comprar" y "Vender"

In [ ]:
#imprimir el DataFrame solo con los dias de Compra y los de Venta

medias_cv = medias[(medias.accion == 'Vender') | (medias.accion == 'Comprar')]  # el simbolo "|" significo "o"
medias_cv.reset_index(inplace = True)
medias_cv

In [ ]:
#calculo la variacion entre las filas "Vender" que tiene como fila anterior a "Comprar"

for i in range(1, len(medias_cv)):

    if (medias_cv.loc[i,'accion'] == 'Vender') and (medias_cv.loc[i-1,'accion'] == 'Comprar'):
        
        medias_cv.loc[i,'variacion'] = (medias_cv.loc[i,'Adj Close'] / medias_cv.loc[i-1,'Adj Close'] - 1) * 100
        
medias_cv

In [ ]:
#desplegamos toda la tabla para ver si está bien calculado

pd.options.display.max_columns = None
pd.options.display.max_rows = None
medias_cv

In [ ]:
#Nos quedamos solo con las filas que tiene calculada la "variacion porcentual"

pd.options.display.max_rows = 20

medias_cv_parcial = medias_cv.dropna()
medias_cv_parcial

In [ ]:
#Hacemos un histograma de la variacion para ver como son las variaciones

medias_cv_parcial.variacion.plot(kind='hist', bins=100)

In [ ]:
print('El promedio es: ', round(medias_cv_parcial.variacion.mean(),2))
print('La mediana es: ', round(medias_cv_parcial.variacion.median(),2))
print('El mínimo es: ', round(medias_cv_parcial.variacion.min(),2))
print('El maximo es: ', round(medias_cv_parcial.variacion.max(),2))
print('*Los datos están en porcentaje')

In [ ]:
#Junto los Retornos con el RSI

var_rsi= pd.concat([medias_cv_parcial['variacion'],medias_cv_parcial['rsi']], axis=1)

var_rsi.plot('variacion', 'rsi', s=1, kind='scatter')

In [ ]:
#Junto los Retornos con el MACD

var_macd= pd.concat([medias_cv_parcial['variacion'],medias_cv_parcial['histograma']], axis=1)

var_macd.plot('variacion', 'histograma', s=1, kind='scatter')